In [1]:
import pandas as pd
import numpy as np
import pprint
from os import walk
import scipy.stats as stats
import matplotlib.pyplot as plt

In [6]:
dna_src = pd.read_excel("/Users/sohamdighe/Documents/SCIE90017/Data/ADNI/Genetic/ADNI_GO_2_GWAS_FinalReports_Keys.xlsx")
dna_src

/Users/sohamdighe/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Column Name,Description,Data Type,Expected Values,Notes
0,Sample Index,Row index of sample,integer,NaN,NaN
1,Sample ID,Sample identifier: ADNI_ID,string,NaN,NaN
2,GC Score,The call score of this SNP for the sample. Gen...,float,0-1 or NaN,NaN: not available or not applicable
3,SNP Index,The row index of the SNP,integer,1-620901,NaN
4,SNP Name,The name of the SNP,string,NaN,NaN
5,Allele1 - Top,Allele 1 on the top strand. Illumina-designate...,character,"A,C,G,T,-","""-"": not available or not applicable"
6,Allele2 - Top,Allele 2 on the top strand. Illumina-designate...,character,"A,C,G,T,-","""-"": not available or not applicable"
7,Allele1 - Forward,Allele 1 on the forward strand. Forward/revers...,character,"A,C,G,T,-","""-"": not available or not applicable"
8,Allele2 - Forward,Allele 2 on the forward strand. Forward/revers...,character,"A,C,G,T,-","""-"": not available or not applicable"
9,Allele1 - AB,Allele 1 in A/B notation,character,"A,B,-","""-"": not available or not applicable"


In [17]:
desikan = pd.read_csv("/Users/sohamdighe/Documents/SCIE90017/Data/ADNI/Genetic/DESIKANLAB.csv")
drop_col=['update_stamp','USERDATE']#1 and 2 unique values so they are not important
desikan.drop(drop_col,axis=1, inplace=True)
desikan.isnull().sum()

RID    0
PHS    0
CIR    0
dtype: int64

In [33]:
tomm40 = pd.read_csv("/Users/sohamdighe/Documents/SCIE90017/Data/ADNI/Genetic/TOMM40.csv")
tomm40.drop(['update_stamp'],axis=1,inplace=True)
#polyT variant data
tomm40.dropna(axis=0,subset=['TOMM40_A1', 'TOMM40_A2'],inplace=True)
tomm40.isnull().sum()

PTID         0
RID          0
TOMM40_A1    0
TOMM40_A2    0
dtype: int64

In [15]:
ts_ratio = pd.read_csv("/Users/sohamdighe/Documents/SCIE90017/Data/ADNI/Genetic/TS_RATIO_ADJ.csv")
ts_ratio.isnull().sum()

RID               0
PHASE             0
PLATE             0
WELL              0
VISCODE           0
VISCODE2         83
EXAM_DATE         2
DATE_DRAWN        0
DATE_RECEIVED    41
DNA_STORAGE       0
SAMPLE_TYPE       0
TS_RATIO          0
CV                0
TS_RATIO_ADJ      0
TL_BP_ADJ         0
update_stamp      0
dtype: int64

In [37]:
ts_ratio[ts_ratio.VISCODE2.isnull()]
#how important is viscode2 and date_received

,RID,PHASE,PLATE,WELL,VISCODE,VISCODE2,EXAM_DATE,DATE_DRAWN,DATE_RECEIVED,DNA_STORAGE,SAMPLE_TYPE,TS_RATIO,CV,TS_RATIO_ADJ,TL_BP_ADJ,update_stamp
30,835,ADNIGO,1,C07,sc,NaN,2011-09-13,2011-09-13,2011-09-14,65.97,Blood,0.77,0.05,0.76,5110.94,2018-11-20 11:04:42.0
31,58,ADNI2,1,C08,m60,NaN,2011-04-27,2011-04-28,2011-04-29,70.57,Blood,0.78,0.02,0.76,5098.58,2018-11-20 11:04:42.0
34,4080,ADNI2,1,C11,bl,NaN,2011-07-12,2011-08-03,2011-08-04,67.33,Blood,0.70,0.02,0.71,4981.31,2018-11-20 11:04:42.0
82,1010,ADNIGO,1,G11,sc,NaN,2010-12-02,2010-12-02,2010-12-03,75.47,Blood,0.81,0.00,0.85,5319.43,2018-11-20 11:04:42.0
135,1271,ADNI2,2,D04,m48,NaN,2011-03-30,2011-04-04,2011-04-05,71.37,Blood,0.96,0.01,1.02,5739.17,2018-11-20 11:04:42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,2037,ADNIGO,19,F09,bl,NaN,2010-08-24,2010-09-01,2010-09-02,78.53,Blood,1.21,0.01,1.23,6231.31,2018-11-20 11:04:43.0
1795,1408,ADNIGO,19,F10,sc,NaN,2010-10-14,2010-10-14,2010-10-15,77.10,Blood,0.73,0.05,0.77,5134.93,2018-11-20 11:04:43.0
1860,4707,ADNI2,20,D03,bl,NaN,2012-06-05,2012-06-05,2012-06-06,57.10,Blood,0.83,0.00,0.82,5262.77,2018-11-20 11:04:43.0
1881,108,ADNIGO,20,E12,sc,NaN,2011-02-15,2011-02-15,2011-02-16,72.97,Blood,0.63,0.03,0.71,4977.29,2018-11-20 11:04:43.0


In [38]:
ts_ratio[ts_ratio.DATE_RECEIVED.isnull()]

,RID,PHASE,PLATE,WELL,VISCODE,VISCODE2,EXAM_DATE,DATE_DRAWN,DATE_RECEIVED,DNA_STORAGE,SAMPLE_TYPE,TS_RATIO,CV,TS_RATIO_ADJ,TL_BP_ADJ,update_stamp
104,2390,ADNI2,2,A09,m12,m12,2012-07-17,2012-07-24,NaN,55.47,Buffy Coat,1.24,0.03,1.23,6244.30,2018-11-20 11:04:42.0
137,230,ADNI2,2,D06,m96,m96,2014-01-14,2014-01-14,NaN,37.50,Buffy Coat,0.67,0.02,0.62,4771.23,2018-11-20 11:04:42.0
152,2390,ADNI2,2,E09,m36,m36,2014-07-17,2014-07-17,NaN,31.37,Buffy Coat,0.78,0.04,0.76,5100.76,2018-11-20 11:04:42.0
173,230,ADNI2,2,G06,m96,m96,2014-01-14,2014-01-14,NaN,37.50,Buffy Coat,0.70,0.01,0.66,4862.78,2018-11-20 11:04:42.0
185,230,ADNI2,2,H06,m84,m84,2013-01-29,2013-01-29,NaN,49.13,Buffy Coat,0.91,0.01,0.89,5429.51,2018-11-20 11:04:42.0
198,4173,ADNI2,3,A07,bl,bl,2011-09-20,2011-09-20,NaN,65.73,blood,0.92,0.04,0.89,5420.24,2018-11-20 11:04:42.0
234,4173,ADNI2,3,D07,m12,m12,2012-09-25,2012-09-25,NaN,53.37,Buffy Coat,1.48,0.05,1.39,6633.29,2018-11-20 11:04:42.0
258,4173,ADNI2,3,F07,m24,m24,2013-09-27,2013-09-27,NaN,41.03,Buffy Coat,1.04,0.00,0.94,5544.94,2018-11-20 11:04:42.0
297,1004,ADNI2,4,A10,m60,m60,2011-12-28,2011-12-28,NaN,62.43,blood,0.67,0.05,0.73,5025.86,2018-11-20 11:04:42.0
333,1004,ADNI2,4,D10,m72,m72,2012-11-09,2012-11-09,NaN,51.73,Buffy Coat,0.99,0.05,0.99,5669.47,2018-11-20 11:04:42.0
